# Exploration & Preprocessing: Mental Health in Tech 2016

## 1. Data Exploration

In [1]:
import pandas as pd
import os

RUN = 'run_02'

df = pd.read_csv('../../data/mental-heath-in-tech-2016_20161114.csv')

print(f"Participants (rows): {df.shape[0]}")
print(f"Variables (columns): {df.shape[1]}")

Participants (rows): 1433
Variables (columns): 63


## 2. Data Type Analysis

In [2]:
# Display data types of all columns as DataFrame
dtype_df = pd.DataFrame({
    'Data Type': df.dtypes.values,
    'Column Name': df.columns
})

display(dtype_df)

,Data Type,Column Name
0,int64,Are you self-employed?
1,object,How many employees does your company or organi...
2,float64,Is your employer primarily a tech company/orga...
3,float64,Is your primary role within your company relat...
4,object,Does your employer provide mental health benef...
...,...,...
58,object,What US state or territory do you live in?
59,object,What country do you work in?
60,object,What US state or territory do you work in?
61,object,Which of the following best describes your wor...


## 3. Outlier Analysis

In [3]:
# Age column analysis
age_col = 'What is your age?'
ages = df[age_col]

# Find suspicious values
print("\nSuspicious values (age < 18 or age > 70):")
suspicious = df[(ages < 18) | (ages > 70)][age_col]
print(suspicious.values)


Suspicious values (age < 18 or age > 70):
[ 17  99 323   3  15  74]


In [4]:
# Remove unrealistic age values
df = df[~df['What is your age?'].isin([3, 99, 323])]

## 4. Data Consistency

In [5]:
# Gender: Show all unique values
gender_col = 'What is your gender?'
print(f"Gender - {df[gender_col].nunique()} unique values:\n")
print(df[gender_col].value_counts())

Gender - 69 unique values:

What is your gender?
Male                                       608
male                                       249
Female                                     153
female                                      95
M                                           86
                                          ... 
female-bodied; no feelings about gender      1
cis man                                      1
AFAB                                         1
Transgender woman                            1
MALE                                         1
Name: count, Length: 69, dtype: int64


In [6]:
import numpy as np

# Normalize gender: 70 variants -> 3 categories (Male / Female / Other)
male_variants = [
    'male', 'm', 'man', 'male ', 'male.', 'malr', 'mail', 'm|', 'dude', 
    'cis male', 'cis man', 'cisdude', 'male (cis)', 'sex is male', 
    'i\'m a man why didn\'t you make this a drop down question. you should of asked sex? and i would of answered yes please. seriously how much text can this take?'
]

female_variants = [
    'female', 'f', 'woman', 'female ', ' female', 'fem', 'fm',
    'female/woman', 'cis female ', 'cis female', 'cisgender female', 
    'cis-woman', 'female assigned at birth ', 'i identify as female.',
    'female (props for making this a freeform field, though)', 'afab'
]

def normalize_gender(val):
    if pd.isna(val):
        return np.nan
    val_lower = str(val).lower().strip()
    if val_lower in male_variants:
        return 'Male'
    elif val_lower in female_variants:
        return 'Female'
    else:
        return 'Other'

df[gender_col] = df[gender_col].apply(normalize_gender)

print("Gender normalized: 70 -> 3 categories\n")
print(df[gender_col].value_counts())

Gender normalized: 70 -> 3 categories

What is your gender?
Male      1055
Female     338
Other       34
Name: count, dtype: int64


## 5. Missing Values

In [7]:
# Missing values per column
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing_pct = (missing / len(df) * 100).round(1)

print(f"Columns with missing values: {len(missing)}\n")
for col in missing.index:
    print(f"{missing[col]:4} {missing_pct[col]:5.1f}% - {col}")

Columns with missing values: 44

1286  89.9% - If you have revealed a mental health issue to a client or business contact, do you believe this has impacted you negatively?
1226  85.7% - If yes, what percentage of your work time (time performing primary or secondary job functions) is affected by a mental health issue?
1167  81.6% - Is your primary role within your company related to tech/IT?
1143  79.9% - Do you know local or online resources to seek help for a mental health disorder?
1143  79.9% - If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to clients or business contacts?
1143  79.9% - If you have revealed a mental health issue to a coworker or employee, do you believe this has impacted you negatively?
1143  79.9% - Do you believe your productivity is ever affected by a mental health issue?
1143  79.9% - If you have been diagnosed or treated for a mental health disorder, do you ever reveal this to coworkers or employees?
1143  79.9% - Do

In [8]:
# Remove columns with >70% missing values
threshold = 0.70
missing_pct = df.isnull().sum() / len(df)
cols_to_drop = missing_pct[missing_pct > threshold].index.tolist()

df = df.drop(columns=cols_to_drop)

# RUN_02: Exclude self-employed participants (Feature Selection)
# Self-employed have "Not applicable" for all employer questions, which
# dominated clustering in run_01. HR measures only apply to employees.
n_before = len(df)
df = df[df['Are you self-employed?'] == 0]
n_after = len(df)

In [9]:
# Remove US State columns (too many categories, only relevant for US participants)
us_state_cols = [
    'What US state or territory do you live in?',
    'What US state or territory do you work in?'
]
df = df.drop(columns=[col for col in us_state_cols if col in df.columns])

In [10]:
# RUN_02: Current employer questions - NaN handling simplified
# Since self-employed are excluded, most NaN values should be gone
# Remaining NaN values are imputed with mode in the next step
current_employer_cols = [
    'How many employees does your company or organization have?',
    'Is your employer primarily a tech company/organization?',
    'Does your employer provide mental health benefits as part of healthcare coverage?',
    'Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?',
    'Does your employer offer resources to learn more about mental health concerns and options for seeking help?',
    'Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources provided by your employer?',
    'Do you know the options for mental health care available under your employer-provided coverage?',
    'If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:',
    'Do you think that discussing a mental health disorder with your employer would have negative consequences?',
    'Do you think that discussing a physical health issue with your employer would have negative consequences?',
    'Would you feel comfortable discussing a mental health disorder with your coworkers?',
    'Would you feel comfortable discussing a mental health disorder with your direct supervisor(s)?',
    'Have you heard of or observed negative consequences for co-workers who have been open about mental health issues in your workplace?',
    'Do you feel that your employer takes mental health as seriously as physical health?'
]

# Check remaining NaN values
for col in current_employer_cols:
    if col in df.columns:
        nan_count = df[col].isna().sum()
        if nan_count > 0:
            print(f"{nan_count} NaN remaining in: {col[:50]}...")

133 NaN remaining in: Do you know the options for mental health care ava...


In [11]:
# RUN_02: Remove Previous Employer columns entirely
# These columns dominated clustering because participants without previous employers
# answered "Not applicable" to all questions, creating an artificial cluster.
# For HR recommendations, current employer questions are more relevant.

previous_employer_cols = [
    'Have your previous employers provided mental health benefits?',
    'Were you aware of the options for mental health care provided by your previous employers?',
    'Did your previous employers ever formally discuss mental health (as part of a wellness campaign or other official communication)?',
    'Did your previous employers provide resources to learn more about mental health issues and how to seek help?',
    'Was your anonymity protected if you chose to take advantage of mental health or substance abuse treatment resources with previous employers?',
    'Do you think that discussing a mental health disorder with previous employers would have negative consequences?',
    'Do you think that discussing a physical health issue with previous employers would have negative consequences?',
    'Would you have been willing to discuss a mental health issue with your direct supervisor(s)?',
    'Would you have been willing to discuss a mental health issue with your previous co-workers?',
    'Did you hear of or observe negative consequences for co-workers with mental health issues in your previous workplaces?',
    'Did you feel that your previous employers took mental health as seriously as physical health?',
    'Do you have previous employers?'
]

# Drop all previous employer columns
cols_to_drop = [col for col in previous_employer_cols if col in df.columns]
df = df.drop(columns=cols_to_drop)

print(f"Removed {len(cols_to_drop)} Previous Employer columns")

Removed 12 Previous Employer columns


In [12]:
# Impute remaining categorical columns with mode
# Exclude free text columns (handled separately)
free_text_cols = ['Why or why not?', 'Why or why not?.1']

# Find columns with remaining NaN values
cols_with_nan = df.columns[df.isnull().any()].tolist()
cols_to_impute = [col for col in cols_with_nan if col not in free_text_cols]

# Impute with mode for categorical columns
for col in cols_to_impute:
    if df[col].dtype == 'object':
        mode_value = df[col].mode()[0]
        df[col] = df[col].fillna(mode_value)

## 6. Multi-Value Columns

In [13]:
# Define category groups for mental health conditions
condition_groups = {
    'anxiety_related': [
        'Anxiety Disorder (Generalized, Social, Phobia, etc)',
        'Obsessive-Compulsive Disorder',
        'Post-traumatic Stress Disorder',
        'Stress Response Syndromes'
    ],
    'mood_related': [
        'Mood Disorder (Depression, Bipolar Disorder, etc)'
    ],
    'neurodevelopmental': [
        'Attention Deficit Hyperactivity Disorder'
    ],
    'other_conditions': [
        'Substance Use Disorder',
        'Addictive Disorder',
        'Eating Disorder (Anorexia, Bulimia, etc)',
        'Personality Disorder (Borderline, Antisocial, Paranoid, etc)',
        'Psychotic Disorder (Schizophrenia, Schizoaffective, etc)'
    ]
}

# Define category groups for work positions
position_groups = {
    'role_developer': [
        'Back-end Developer',
        'Front-end Developer'
    ],
    'role_ops': [
        'DevOps/SysAdmin',
        'Support'
    ],
    'role_leadership': [
        'Supervisor/Team Lead',
        'Executive Leadership'
    ],
    'role_other': [
        'Designer',
        'Dev Evangelist/Advocate',
        'One-person shop',
        'Sales',
        'Other'
    ]
}


In [14]:
def grouped_one_hot_encode(df, column, groups, prefix):
    """
    One-hot encode a multi-value column using predefined groups.
    Uses vectorized string operations for better performance.
    """
    # Create binary column for each group
    for group_name, categories in groups.items():
        col_name = f"{prefix}_{group_name}"
        
        # Use vectorized str.contains with regex pattern
        pattern = '|'.join([cat.replace('(', r'\(').replace(')', r'\)') for cat in categories])
        df[col_name] = df[column].str.contains(pattern, na=False, regex=True).astype(int)
    
    # Drop original column
    df = df.drop(columns=[column])
    
    return df

# Mental health diagnosis columns (2 remaining after missing value cleanup)
mental_health_cols = [
    ('If yes, what condition(s) have you been diagnosed with?', 'diagnosed_yes'),
    ('If so, what condition(s) were you diagnosed with?', 'diagnosed_pro')
]
# Note: "If maybe..." was removed due to >70% missing values

for col, prefix in mental_health_cols:
    df = grouped_one_hot_encode(df, col, condition_groups, prefix)

# Apply to work position column
df = grouped_one_hot_encode(
    df, 
    'Which of the following best describes your work position?',
    position_groups,
    'position'
)

## 7. Ordinal Encoding

In [15]:
# Ordinal Encoding: Convert ordered categories to numbers

# 1. Company size
df['How many employees does your company or organization have?'] = df['How many employees does your company or organization have?'].map({
    '1-5': 1, '6-25': 2, '26-100': 3, '100-500': 4, '500-1000': 5, 'More than 1000': 6, 'Not applicable': 0
})

# 2. Medical leave difficulty  
df['If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'] = df['If a mental health issue prompted you to request a medical leave from work, asking for that leave would be:'].map({
    'Very easy': 1, 'Somewhat easy': 2, 'Neither easy nor difficult': 3, 'Somewhat difficult': 4, 'Very difficult': 5, "I don't know": 3, 'Not applicable': 0
})

# 3. Openness to share
df['How willing would you be to share with friends and family that you have a mental illness?'] = df['How willing would you be to share with friends and family that you have a mental illness?'].map({
    'Not open at all': 1, 'Somewhat not open': 2, 'Neutral': 3, 'Somewhat open': 4, 'Very open': 5,
    'Not applicable to me (I do not have a mental illness)': 0
})

# 4. Remote work
df['Do you work remotely?'] = df['Do you work remotely?'].map({
    'Never': 0, 'Sometimes': 1, 'Always': 2
})

# 5. Work interference when treated
df['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'] = df['If you have a mental health issue, do you feel that it interferes with your work when being treated effectively?'].map({
    'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Often': 3, 'Not applicable to me': 0
})

# 6. Work interference when NOT treated
df['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'] = df['If you have a mental health issue, do you feel that it interferes with your work when NOT being treated effectively?'].map({
    'Never': 0, 'Rarely': 1, 'Sometimes': 2, 'Often': 3, 'Not applicable to me': 0
})

print("Ordinal encoding applied to 6 columns")

Ordinal encoding applied to 6 columns


## 8. Nominal Encoding

In [16]:
# Country columns: Binary encoding (USA vs. Non-USA)

df['country_live_usa'] = (df['What country do you live in?'] == 'United States of America').astype(int)
df['country_work_usa'] = (df['What country do you work in?'] == 'United States of America').astype(int)

# Drop original columns
df = df.drop(columns=['What country do you live in?', 'What country do you work in?'])

In [17]:
# One-Hot Encoding for all remaining categorical columns
free_text_cols = ['Why or why not?', 'Why or why not?.1']

# Get all remaining categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
cols_to_encode = [col for col in categorical_cols if col not in free_text_cols]

df = pd.get_dummies(df, columns=cols_to_encode, drop_first=False, dtype=int)

## 9. Free Text Handling

In [18]:
# Remove free text columns (not used for clustering)
free_text_cols = ['Why or why not?', 'Why or why not?.1']
df = df.drop(columns=[col for col in free_text_cols if col in df.columns])

## 10. Save Data

In [19]:
# Save preprocessed data
output_dir = f'../../data/{RUN}/processed'
os.makedirs(output_dir, exist_ok=True)
output_path = f'{output_dir}/mental_health_preprocessed.csv'
df.to_csv(output_path, index=False)

print(f"Data saved to: {output_path}")
print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")

Data saved to: ../../data/run_02/processed/mental_health_preprocessed.csv
Shape: 1143 rows, 94 columns
